расчет var, cvar для удержания позиции по spy в течение 10 минут.
var(value at risk) говорит "в x% доход будет выше этого значения".
cvar(conditional value at risk) говорит "когда будет неудача (т.е. случится неудача, которая появляется с вероятностью (1-x)%), то средний убыток вот это значение".

In [ ]:

# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Загружаем NPZ файл с минутными данными SPXUSD
file_path = "../npz/SPXUSD_M1_2017.npz"
data = np.load(file_path, allow_pickle=True)
spy_data = data['data']

# Преобразуем в DataFrame
columns = ["timestamp", "close"]
df_spy = pd.DataFrame(spy_data, columns=columns)

# Преобразуем timestamp в datetime
df_spy["timestamp"] = pd.to_datetime(df_spy["timestamp"])

# Проверяем тип данных в столбце "close"
df_spy["close"] = pd.to_numeric(df_spy["close"], errors="coerce")

# Считаем лог-доходности (минутные) заново
df_spy["log_return"] = np.log(df_spy["close"] / df_spy["close"].shift(10))

# Убираем NaN, возникшие при сдвиге
df_spy = df_spy.dropna()

# Рассчитываем метрики
expected_return_spy = df_spy["log_return"].mean()
std_dev_spy = df_spy["log_return"].std()

# 95%-VaR (по историческому методу)
var_95_spy = df_spy["log_return"].quantile(0.05)

# 95%-CVaR (средний убыток в худших 5% случаев)
cvar_95_spy = df_spy["log_return"][df_spy["log_return"] <= var_95_spy].mean()

# Визуализируем распределение доходностей
plt.figure(figsize=(12, 6))
plt.hist(df_spy["log_return"], bins=50, alpha=0.7, color='blue', edgecolor='black', label='SPXUSD Log Returns')
plt.yscale("log")
plt.axvline(expected_return_spy, color='black', linestyle='dashed', linewidth=2, label=f'Mean: {expected_return_spy:.6f}')
plt.axvline(var_95_spy, color='red', linestyle='dashed', linewidth=2, label=f'95%-VaR: {var_95_spy:.6f}')
plt.axvline(cvar_95_spy, color='orange', linestyle='dashed', linewidth=2, label=f'95%-CVaR: {cvar_95_spy:.6f}')
plt.title("Histogram of SPXUSD Minute Log Returns")
plt.xlabel("Log Return")
plt.ylabel("Frequency")
plt.legend()
plt.grid()


plt.show()

# Вывод результатов
expected_return_spy, var_95_spy, cvar_95_spy
